In [6]:
# Import libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

In [7]:
def clean_names(names):
    return names.str.lower().str.replace(' ', '').str.replace('-','')

# Top 100 mineable coins

In [8]:
# Top 100 mineable coins
url = 'https://coinmarketcap.com/coins/views/filter-non-mineable/'

r = requests.get(url) 

page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')
table = soup.find_all('div', class_='cmc-table__table-wrapper-outer')

In [9]:
# Get column names
column_names = []
for header in table[0].find_all('th'):
    column_names.append(str(header.string))
# Drop graphs
column_names.pop()
column_names.pop()

data = []
for row in table[2].find_all('tr')[1:]:
    new_row = []
    for entry in row:
        new_row.append(entry.text)
    new_row.pop()
    new_row.pop()
    data.append(new_row)
mineable_100 = pd.DataFrame(data, columns = column_names)
mineable_100 = mineable_100.set_index('Name')
mineable_100 = mineable_100.set_index(clean_names(mineable_100.index))
mineable_100 = mineable_100[['#', "Market Cap", "Price"]]
mineable_100 = mineable_100.rename(columns={'Market Cap': 'market_cap', 'Price':'price'})

# What to mine, nethash

## ASIC

In [10]:
url = 'https://whattomine.com/asic.json'

r = requests.get(url) 
data = r.json()

whattomine_asic = pd.DataFrame(data)
# normalize + clean names
whattomine_asic = pd.json_normalize(whattomine_asic['coins']).set_index(clean_names(whattomine_asic.index.str.lower()))
whattomine_asic['block_time'] = pd.to_numeric(whattomine_asic['block_time'])
whattomine_asic['nethash'] = pd.to_numeric(whattomine_asic['nethash'])
# Drop some columns
whattomine_asic = whattomine_asic[['tag','algorithm', 'block_time', 'difficulty', 'nethash', 'block_reward', 'exchange_rate', 'exchange_rate_curr']]
whattomine_asic[:4]

,tag,algorithm,block_time,difficulty,nethash,block_reward,exchange_rate,exchange_rate_curr
actinium,ACM,Lyra2z,144.0,3.305795e+03,9.859917e+10,50.0000,0.000002,BTC
axe,AXE,X11,154.0,1.968569e+06,5.490220e+13,1.9400,0.000037,BTC
bitcoin,BTC,SHA-256,648.0,1.471521e+13,9.753297e+19,12.6872,7049.610000,BTC
bitcoincashabc,BCH,SHA-256,544.0,2.596584e+11,2.050044e+18,6.2500,0.032851,BTC


## GPU

In [11]:
url = 'https://whattomine.com/coins.json'

r = requests.get(url) 
data = r.json()

whattomine_gpu = pd.DataFrame(data)
whattomine_gpu = pd.json_normalize(whattomine_gpu['coins']).set_index(clean_names(whattomine_gpu.index.str.lower()))
whattomine_gpu['block_time'] = pd.to_numeric(whattomine_gpu['block_time'])
whattomine_gpu['nethash'] = pd.to_numeric(whattomine_gpu['nethash'])
# Drop some columns
whattomine_gpu = whattomine_gpu[['tag','algorithm', 'block_time', 'difficulty', 'nethash', 'block_reward', 'exchange_rate', 'exchange_rate_curr']]
whattomine_gpu[:4]

,tag,algorithm,block_time,difficulty,nethash,block_reward,exchange_rate,exchange_rate_curr
aeternity,AE,CuckooCycle,179.0,2.883950e+07,161114,177.30900,1.438000e-05,BTC
aion,AION,"Equihash (210,9)",11.0,1.135886e+07,1032623,4.50000,9.060000e-06,BTC
anon,ANON,EquihashZero,152.0,2.764322e+05,1818,6.53125,8.400000e-07,BTC
beam,BEAM,BeamHashII,60.0,1.044512e+08,1740852,40.00000,3.823000e-05,BTC


# CoinWarz

In [12]:
url = 'https://www.coinwarz.com/v1/api/profitability?apikey=58467a6f2e2d4c65b7ef720b26137453&algo=all'
r = requests.get(url) 
data = r.json()

coinwarz = pd.DataFrame(data)
    
coinwarz = pd.json_normalize(coinwarz['Data'])
coinwarz = coinwarz.set_index(clean_names(coinwarz['CoinName']))

# Drop and rename some columns
coinwarz = coinwarz[['CoinTag', 'Algorithm','BlockTimeInSeconds', 'Difficulty', 'BlockReward', 'ExchangeRate']]
coinwarz = coinwarz.rename(columns={'CoinTag': 'tag', 'Algorithm': 'algorithm','BlockTimeInSeconds':'block_time',
                                    'Difficulty': 'difficulty', 'BlockReward': 'block_reward', 'ExchangeRate': 'exchange_rate'})
coinwarz['nethash_scraped'] = 0
coinwarz[:4]

,tag,algorithm,block_time,difficulty,block_reward,exchange_rate,nethash_scraped
CoinName,,,,,,,
litecoincash,LCC,SHA-256,75,8.729129e+06,125.0,5.200000e-07,0
ethereumclassic,ETC,EtHash,15,1.147544e+14,4.0,7.616000e-04,0
ethereum,ETH,EtHash,15,2.209919e+15,2.0,2.415566e-02,0
cannabiscoin,CANN,X11,42,2.480494e+04,70.0,2.900000e-07,0


In [13]:
coinwarz.loc['bitcoin']

tag                        BTC
algorithm              SHA-256
block_time                 600
difficulty         1.47152e+13
block_reward              12.5
exchange_rate          7070.51
nethash_scraped              0
Name: bitcoin, dtype: object

Here, the exchange rate is relative the bitcoin, except for bitcoin itself, where it is in US$

# Combining the data

In [14]:
def check_missing(data):
    missing_data = result[data.isnull().any(axis=1)]
    return missing_data.shape[0]

In [15]:
whattomine = whattomine_gpu.append(whattomine_asic)
whattomine = whattomine.rename(columns={'nethash':'nethash_scraped'})
whattomine

,tag,algorithm,block_time,difficulty,nethash_scraped,block_reward,exchange_rate,exchange_rate_curr
aeternity,AE,CuckooCycle,179.0,2.883950e+07,1.611140e+05,177.30900,1.438000e-05,BTC
aion,AION,"Equihash (210,9)",11.0,1.135886e+07,1.032623e+06,4.50000,9.060000e-06,BTC
anon,ANON,EquihashZero,152.0,2.764322e+05,1.818000e+03,6.53125,8.400000e-07,BTC
beam,BEAM,BeamHashII,60.0,1.044512e+08,1.740852e+06,40.00000,3.823000e-05,BTC
bitcash,BITC,X25X,63.0,2.100000e+01,1.431656e+09,19.35000,5.900000e-07,BTC
...,...,...,...,...,...,...,...,...
vergegroestl,XVG,Myr-Groestl,150.0,9.355731e+06,2.678837e+14,400.00000,3.800000e-07,BTC
vergelyra2rev2,XVG,Lyra2REv2,150.0,9.784490e+04,2.801604e+12,400.00000,3.800000e-07,BTC
vergescrypt,XVG,Scrypt,150.0,1.210623e+04,3.466390e+11,400.00000,3.800000e-07,BTC
viacoin,VIA,Scrypt,24.0,3.053552e+05,5.464544e+13,0.62500,1.722000e-05,BTC


In [16]:
result = mineable_100.join(whattomine, how="left")

In [17]:
for coin in coinwarz.index:
    if coin not in whattomine.index.values:
        if coin in result.index:
            result.loc[coin, ['tag','algorithm','block_time','difficulty','nethash_scraped']] = coinwarz.loc[coin]
check_missing(result)

67

In [18]:
result[:10]

,#,market_cap,price,tag,algorithm,block_time,difficulty,nethash_scraped,block_reward,exchange_rate,exchange_rate_curr
Name,,,,,,,,,,,
bitcoin,1,"$129,897,986,712","$7,086.58",BTC,SHA-256,648.0000,1.471521e+13,9.753297e+19,12.687200,7049.610000,BTC
ethereum,2,"$18,899,319,373",$170.95,ETH,Ethash,13.5073,2.206678e+15,1.633693e+14,2.000000,0.024131,BTC
bitcoincash,3,"$4,297,081,457",$233.76,BCH,SHA-256,600.0000,2.596584e+11,0.000000e+00,NaN,NaN,NaN
bitcoinsv,4,"$3,580,454,933",$194.79,BSV,SHA-256,554.0000,2.312089e+11,1.792482e+18,6.250000,0.027495,BTC
litecoin,5,"$2,741,701,798",$42.49,LTC,Scrypt,134.0000,5.326059e+06,1.707108e+14,12.500000,0.005976,BTC
monero,6,"$991,687,990",$56.58,XMR,RandomX,116.0000,1.557296e+11,1.342497e+09,1.756834,0.007995,BTC
cardano,7,"$893,573,496",$0.034465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tron,8,"$861,568,024",$0.012921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dash,9,"$712,230,035",$75.42,DASH,X11,157.0000,2.051579e+08,5.612398e+15,1.553313,0.010628,BTC


In [42]:
result.to_csv('data/mineable_100.csv')

In [25]:
found_data = result[~result.isnull().any(axis=1)]
found_data['market_cap']

'$129,897,986,712$18,899,319,373$3,580,454,933$2,741,701,798$991,687,990$712,230,035$627,724,640$345,876,857$248,480,865$174,305,241$140,883,718$100,087,931$98,055,918$77,977,356$70,281,311$58,106,547$48,597,671$45,661,555$34,396,020$31,335,419$26,014,526$16,303,930$13,223,291$11,386,836$8,590,262$4,670,124$3,366,495$2,905,590$2,854,383$2,660,166$2,560,488$2,478,207$1,863,332'